In [1]:
import os 
os.chdir('/home/ariel/dev/odds_data/')

import pandas as pd 
from tqdm import tqdm
from utils.odds_api_accessor import OddsAccessor
from database_config import current_config, DB_URI
from datetime import datetime
import pytz
from sqlalchemy import create_engine
from sqlalchemy.exc import IntegrityError
import numpy as np

TIMEZONE = pytz.timezone('America/New_York')
oddsapi = OddsAccessor(current_config.get('_20K_API_KEY'))
date_range_list = pd.date_range(start='2021-01-01', end='2025-03-30')
engine = create_engine(DB_URI)

In [12]:
import json

In [20]:
with open('hist_main_odds_results.json', 'r') as f:
    hist_results = json.load(f)

In [21]:
def flatten_odds_data(hist_results):
    # Initialize empty lists to store flattened data
    flattened_data = []
    
    # Extract timestamp info
    timestamp = hist_results['timestamp']
    previous_timestamp = hist_results['previous_timestamp']
    next_timestamp = hist_results['next_timestamp']
    
    # Iterate through each event in the data
    for event in hist_results['data']:
        event_id = event['id']
        sport_key = event['sport_key']
        sport_title = event['sport_title']
        commence_time = event['commence_time']
        home_team = event['home_team']
        away_team = event['away_team']
        
        # Iterate through bookmakers
        for bookmaker in event['bookmakers']:
            bookmaker_key = bookmaker['key']
            bookmaker_title = bookmaker['title']
            bookmaker_last_update = bookmaker['last_update']
            
            # Iterate through markets
            for market in bookmaker['markets']:
                market_key = market['key']
                market_last_update = market['last_update']
                
                # Iterate through outcomes
                for outcome in market['outcomes']:
                    row = {
                        'timestamp': timestamp,
                        'previous_timestamp': previous_timestamp,
                        'next_timestamp': next_timestamp,
                        'id': event_id,
                        'sport_key': sport_key,
                        'sport_title': sport_title,
                        'commence_time': commence_time,
                        'home_team': home_team,
                        'away_team': away_team,
                        'bookmaker_key': bookmaker_key,
                        'bookmaker_title': bookmaker_title,
                        'bookmaker_last_update': bookmaker_last_update,
                        'market_key': market_key,
                        'market_last_update': market_last_update,
                        'outcome_name': outcome['name'],
                        'outcome_price': outcome['price'],
                        'outcome_point': outcome.get('point', None)  # Some markets (like h2h) don't have points
                    }
                    flattened_data.append(row)
    
    # Create DataFrame from flattened data
    df = pd.DataFrame(flattened_data)
    
    # Convert timestamp columns to datetime
    timestamp_cols = ['timestamp', 'previous_timestamp', 'next_timestamp', 
                     'commence_time', 'bookmaker_last_update', 'market_last_update']
    for col in timestamp_cols:
        df[col] = pd.to_datetime(df[col])
        
    return df

# Use the function
df = flatten_odds_data(hist_results)

In [23]:
df.columns.tolist()

['timestamp',
 'previous_timestamp',
 'next_timestamp',
 'id',
 'sport_key',
 'sport_title',
 'commence_time',
 'home_team',
 'away_team',
 'bookmaker_key',
 'bookmaker_title',
 'bookmaker_last_update',
 'market_key',
 'market_last_update',
 'outcome_name',
 'outcome_price',
 'outcome_point']